In [28]:
import pychor
import galois
import pandas as pd
import numpy as np
from dataclasses import dataclass

p = 2**31-1
GF = galois.GF(p)
p1 = pychor.Party('p1')
p2 = pychor.Party('p2')
dealer = pychor.Party('dealer')

@pychor.local_function
def share(x):
    s1 = GF.Random()
    s2 = GF(x) - s1
    return s1, s2

def deal_shares(x):
    s1, s2 = share(x).untup(2)
    s1.send(dealer, p1)
    s2.send(dealer, p2)
    return s1, s2

def deal_triple():
    # Step 1: generate a, b, c
    a = dealer.constant(GF.Random())
    b = dealer.constant(GF.Random())
    c = a * b

    # Step 2: secret share a, b, c
    a1, a2 = deal_shares(a)
    b1, b2 = deal_shares(b)
    c1, c2 = deal_shares(c)
    return (a1, a2), (b1, b2), (c1, c2)

def protocol_mult(x, y, triple):
    x1, x2 = x
    y1, y2 = y
    (a1, a2), (b1, b2), (c1, c2) = triple

    # Step 1. P1 computes d_1 = x_1 - a_1 and sends the result to P2
    d1 = x1 - a1
    d1.send(p1, p2)

    # Step 2. P2 computes d_2 = x_2 - a_2 and sends the result to P1
    d2 = x2 - a2
    d2.send(p2, p1)

    # Step 3: P1 and P2 both compute $d = d_1 + d_2 = x - a$
    d = d1 + d2

    # Step 4. P1 computes e_1 = y_1 - b_1 and sends the result to P2
    e1 = y1 - b1
    e1.send(p1, p2)

    # Step 5. P2 computes e_2 = y_2 - b_2 and sends the result to P1
    e2 = y2 - b2
    e2.send(p2, p1)

    # Step 6. P1 and P2 both compute $e = e_1 + e_2 = y - b$
    e = e1 + e2

    # Step 7. P1 computes r_1 = d*e + d*b_1 + e*a_1 + c_1
    r_1 = d * e + d * b1 + e * a1 + c1

    # Step 8. P2 computes r_2 = 0 + d*b_2 + e*a_2 + c_2
    r_2 = d * b2 + e * a2 + c2

    return r_1, r_2


# Building Applications with MPC

We saw a very simple application of MPC in Chapter 2 (counting the number of patients with heart disease, without sharing the underlying data), and we've now seen protocols for doing both addition and multiplication. What kinds of applications can we build with these tools, and how can we design APIs that make it easy to do?

Throughout this book, we'll use a style of encapsulating secret-shared data with *special objects* and defining operations on those objects using the fundamental MPC protocols we've learned about. For example, we'll start this chapter by defining a `SecInt` class for holding secret-shared integers, and we'll define addition and multiplication for these objects using the additive homomorphism and multiplication triples, respectively. This approach enables building applications that look very similar to traditional centralized Python programs, so that we don't need to worry about the details of the protocol when building applications.

An alternative style, used commonly in the MPC literature, is to define MPC protocols that process *circuits*, and then to build applications by defining circuits. The circuits used in MPC are similar to hardware circuits, but have only addition and multiplication gates, and each wire in the circuit represents a secret-shared value. We'll cover circuits in detail in Chapter TODO.

## `SecInt`: Secure Integers

We define the `SecInt` class as an abstract representation of secret-shared integers. We'll use Python's [`dataclass`](https://docs.python.org/3/library/dataclasses.html) decorator, since `SecInt` is a container for secret-shared data; the class will have two fields `s1` and `s2` to hold the additive secret shares defining the represented integer. We'll also define several methods:

- The `__add__` method will add two `SecInt` objects by adding the respective secret shares
- The `__mul__` method will multiply two `SecInt` objects using the `protocol_mult` protocol defined in Chapter 4
- The `reveal` method will reveal the value of the `SecInt` object by broadcasting the shares and reconstructing the secret
- The `input` class method will construct at new `SecInt` object by secret sharing the input value

The definition of the class appears below.

In [29]:
@dataclass
class SecInt:
    # s1 is p1's share of the value, and s2 is p2's share
    s1: galois.GF
    s2: galois.GF

    @classmethod
    def input(cls, val):
        """Secret share an input: p1 holds s1, and p2 holds s2"""
        s1, s2 = share(val).untup(2)
        if p1 in val.parties:
            s2.send(p1, p2)
            return SecInt(s1, s2)
        else:
            s1.send(p2, p1)
            return SecInt(s1, s2)

    def __add__(x, y):
        """Add two SecInt objects using local addition of shares"""
        return SecInt(x.s1 + y.s1, x.s2 + y.s2)

    def __mul__(x, y):
        """Multiply two SecInt objects using a triple"""
        triple = multiplication_triples.pop()
        r1, r2 = protocol_mult((x.s1, x.s2), (y.s1, y.s2), triple)
        return SecInt(r1, r2)

    def reveal(self):
        """Reveal the secret value by broadcast and reconstruction"""
        self.s1.send(p1, p2)
        self.s2.send(p2, p1)
        return self.s1 + self.s2

Now we can write choreographic programs describing protocols that operate on `SecInt` objects as if they're regular Python numbers. For example, below a program that computes $(x+y)^3$ without revealing $x$ or $y$. Note that every multiplication of `SecInt` objects consumes a multiplication triple, so our program needs to begin by populating the global pool of multiplication triples as a preprocessing step.

In [30]:
multiplication_triples = []

with pychor.LocalBackend():
    x_input = p1.constant(3)
    y_input = p2.constant(4)

    # Preprocessing: deal multiplication triples
    for _ in range(20):
        multiplication_triples.append(deal_triple())

    # Create secret shares of the inputs
    x = SecInt.input(x_input)
    y = SecInt.input(y_input)

    # Online phase: compute (x+y)^3
    z = x + y
    result = z*z*z
    print('(x+y)^3:', result.reveal())

(x+y)^3: 343@{p1, p2}


## Application: Cross-Tabulation


One common analysis of datasets that involves only addition of integers is *[cross tabulation](https://en.wikipedia.org/wiki/Contingency_table)*, the process of counting the number of people in the dataset with particular combinations of attributes. In our heart disease example, we might want to answer a question like: "do people who experience exercise-induced angina more commonly have heart disease?" To answer this question, we can count how many people have:

- Neither exercise-induced angina nor heart disease
- Exercise-induced angina, but not heart disease
- No exercise-induced angina, but heart disease
- Both exercise-induced angina and heart disease

The results will allow comparing the groups to determine the link between the two.

In our framework, the easiest way to do this is to use the [Pandas function for cross tabulation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html) and then convert the values to a tuple. This approach allows us to separate the values in the tuple and secret share each one of them. The following function builds the contingency table locally, using a single dataset:

In [31]:
@pychor.local_function
def heart_disease_crosstab(df):
    return tuple(pd.crosstab(df['exang'], df['target']).to_numpy().flatten())

Now we can write a program to read in separate datasets for P1 and P2, secret share the associated contingency tables, and add corresponding values:

In [33]:
with pychor.LocalBackend():
    # Read datasets
    df1 = pychor.locally(pd.read_csv, 'heart1.csv'@p1)
    df2 = pychor.locally(pd.read_csv, 'heart2.csv'@p2)

    # Create crosstabs
    crosstab1 = heart_disease_crosstab(df1).untup(4)
    crosstab2 = heart_disease_crosstab(df2).untup(4)

    # Secret share crosstabs
    sec_crosstab1 = [SecInt.input(v) for v in crosstab1]
    sec_crosstab2 = [SecInt.input(v) for v in crosstab2]

    # Add corresponding values and reveal results
    crosstab = [(x + y).reveal() for x, y in zip(sec_crosstab1, sec_crosstab2)]
    crosstab_np = np.array(crosstab).reshape((2,2))
    print('Final crosstab:')
    print(crosstab_np)

Final crosstab:
[[39@{p1, p2} 95@{p1, p2}]
 [49@{p1, p2} 17@{p1, p2}]]
